### Google data EDA

From Google we have 2 tables: 
- company_profiles_google_maps:
    - General data about each company
    - google_id is the primary key
- customer_reviews_google
    - Pretty self explanatory, holds reviews from google about each company
    - also uses google_id as key


## Starting with company_profiles_google_maps

In [1]:
import pandas as pd
import psycopg2 as pg

connection = pg.connect(database='clever',user='clever',host='postgres_clever',port='5432',password='clever')
df_google = pd.read_sql('select * from company_profiles_google_maps',connection)

/tmp/ipykernel_198/1461939918.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_google = pd.read_sql('select * from company_profiles_google_maps',connection)


In [2]:
df_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   google_id                 5865 non-null   object 
 1   name                      5865 non-null   object 
 2   site                      4499 non-null   object 
 3   subtypes                  5858 non-null   object 
 4   type                      5858 non-null   object 
 5   category                  5858 non-null   object 
 6   phone                     5622 non-null   object 
 7   full_address              5865 non-null   object 
 8   borough                   3823 non-null   object 
 9   street                    5845 non-null   object 
 10  city                      5865 non-null   object 
 11  postal_code               5864 non-null   object 
 12  state                     5865 non-null   object 
 13  us_state                  5865 non-null   object 
 14  country 

### Analysis by location

Since the task is focused on making ranks by location, let's start looking at the companies locations.

In [4]:
#Analyzing number of companies in each country/state and trying to determine if there are companies from multiple countries
df_google[['country','state']].groupby(by=['country','state']).size()

country                   state         
United States of America  Arizona              1
                          FL                  51
                          Florida           2993
                          GA                   2
                          Georgia             72
                          Indiana              1
                          North Carolina       6
                          OR                   1
                          Oklahoma            16
                          Oregon              40
                          Pennsylvania         3
                          TX                  11
                          Texas             1521
                          WA                   5
                          Washington        1105
dtype: int64

In [6]:
#Analyzing number of companies in each state but with us_state
df_google[['country','us_state']].groupby(by=['country','us_state']).size()

country                   us_state      
United States of America  Arizona              1
                          Florida           3044
                          Georgia             74
                          Indiana              1
                          North Carolina       6
                          Oklahoma            16
                          Oregon              41
                          Pennsylvania         3
                          Texas             1532
                          Washington        1110
dtype: int64

It seems that us_state is already standardized when comparing with state, so I'll probably use it.

There are only US companies, so I'll leave out the country for the analysis.

In [8]:
#Analyzing state/city
df_google[['us_state','city']].groupby(by=['us_state','city']).size().sort_values(ascending=False)

us_state        city   
Florida         Miami      3079
Texas           Dallas     1533
Washington      Seattle    1111
Georgia         Dallas       74
Oregon          Dallas       41
Oklahoma        Miami        16
North Carolina  Dallas        6
Pennsylvania    Dallas        3
Arizona         Miami         1
Indiana         Miami         1
dtype: int64

Had to look for some cities in some states thinking it was a mistake, but there actually are other Miamis and other Dallas =)

Since this table has a large set of descriptive variables on each company, I'll start looking at the customer_reviews_google table for some metrics

## customer_reviews_google table

In [15]:
df_reviews = pd.read_sql('select * from customer_reviews_google',connection)

/tmp/ipykernel_198/3577837379.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reviews = pd.read_sql('select * from customer_reviews_google',connection)


In [16]:
df_reviews.head()

,google_id,review_id,name,place_id,location_link,reviews_link,reviews,rating,review_pagination_id,author_link,...,review_photo_ids,owner_answer,owner_answer_timestamp,owner_answer_timestamp_datetime_utc,review_link,review_rating,review_timestamp,review_datetime_utc,review_likes,reviews_id
0,0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b,ChZDSUhNMG9nS0VJQ0FnSUNabHRtNEZREAE,Wildcat Movers - Dallas,ChIJb_75o8gmTIYRi4oDGe4uX58,https://www.google.com/maps/place/Wildcat+Move...,https://search.google.com/local/reviews?placei...,1692.0,4.9,CAESBkVnSUliZw==,https://www.google.com/maps/contrib/1180290970...,...,AF1QipNLYp-Of9ChXtPBvekfgNbVJl1122ft9cERq3OW,None,NaN,None,https://www.google.com/maps/reviews/data=!4m8!...,5.0,1.694300e+09,9/9/2023 22:50,NaN,-6.962790e+18
1,0x864e99d71a97da87:0xacad6516b3a8297,ChdDSUhNMG9nS0VJQ0FnSUNabHBhZGtBRRAB,AB Moving,ChIJh9qXGteZToYRl4I6a1HWygo,https://www.google.com/maps/place/AB+Moving/@3...,https://search.google.com/local/reviews?placei...,724.0,4.5,CAESB0VnTUl0UUU=,https://www.google.com/maps/contrib/1103082439...,...,None,None,NaN,None,https://www.google.com/maps/reviews/data=!4m8!...,5.0,1.694297e+09,9/9/2023 22:00,NaN,7.776700e+17
2,0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b,ChZDSUhNMG9nS0VJQ0FnSUNaNXVfUVF3EAE,Wildcat Movers - Dallas,ChIJb_75o8gmTIYRi4oDGe4uX58,https://www.google.com/maps/place/Wildcat+Move...,https://search.google.com/local/reviews?placei...,1692.0,4.9,CAESBkVnSUlidw==,https://www.google.com/maps/contrib/1053763761...,...,AF1QipOIHe2CliGnz_e7zwQ_NcL53Vm9ZVDRVpoxfwGM,None,NaN,None,https://www.google.com/maps/reviews/data=!4m8!...,5.0,1.694292e+09,9/9/2023 20:36,NaN,-6.962790e+18
3,0x549015ed168dab6d:0x301820ce37c9de5b,ChdDSUhNMG9nS0VJQ0FnSUNaNXRQRnZnRRAB,Pure Moving Company Seattle Movers Local & Lon...,ChIJbauNFu0VkFQRW97JN84gGDA,https://www.google.com/maps/place/Pure+Moving+...,https://search.google.com/local/reviews?placei...,597.0,5.0,CAESB0VnTUl1QUU=,https://www.google.com/maps/contrib/1055784996...,...,AF1QipOv4l2y4IL8lsmGXKrOgtpMFSn5FFl_9ejPX6LG,None,NaN,None,https://www.google.com/maps/reviews/data=!4m8!...,5.0,1.694290e+09,9/9/2023 20:05,NaN,3.465560e+18
4,0x549015ed168dab6d:0x301820ce37c9de5b,ChZDSUhNMG9nS0VJQ0FnSUNaNXJTaE93EAE,Pure Moving Company Seattle Movers Local & Lon...,ChIJbauNFu0VkFQRW97JN84gGDA,https://www.google.com/maps/place/Pure+Moving+...,https://search.google.com/local/reviews?placei...,597.0,5.0,CAESB0VnTUl1UUU=,https://www.google.com/maps/contrib/1121212537...,...,"AF1QipOYup8n4bncleRIojnuHjJ3dCToJQJzyOL2eBWj, ...",None,NaN,None,https://www.google.com/maps/reviews/data=!4m8!...,5.0,1.694283e+09,9/9/2023 18:07,NaN,3.465560e+18


It seems that each row here is one review but there are also informations about the company's general "score"

In [20]:
df_reviews['google_id'].value_counts()

google_id
0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b    1692
0x5490157655555555:0x673e5a56019f797e    1500
0x549041c0f9681f39:0x3727a97e3c3fe947     811
0x864e99d71a97da87:0xacad6516b3a8297      724
0x864c23dad897733d:0x2de847b08be4e483     688
                                         ... 
0x864c23be034c2355:0xa6a65be391b3478c       2
0x88f53813f20268c9:0xac132b235c84c47f       1
0x88d9b17e808442cd:0xe6659fb8f7360fb6       1
0x549040fa32f122f9:0x5fe0fd50663e967f       1
0x864e9d445b6632e3:0x52bbf3d66c431198       1
Name: count, Length: 86, dtype: int64

In [23]:
df_reviews['google_id'].nunique()

86

this shows that only a few of the companies actually have reviews

In [26]:
df_reviews[df_reviews['google_id'] == '0x864c23dad897733d:0x2de847b08be4e483'].nunique()

google_id                                1
review_id                              688
name                                     1
place_id                                 1
location_link                            1
reviews_link                             1
reviews                                  1
rating                                   1
review_pagination_id                   687
author_link                            688
author_title                           688
author_id                              674
author_image                           688
author_reviews_count                    55
review_text                            458
review_img_url                           3
review_img_urls                          3
review_questions                         0
review_photo_ids                         3
owner_answer                             0
owner_answer_timestamp                   0
owner_answer_timestamp_datetime_utc      0
review_link                            688
review_rati

From this it seems that the rating, reviews and company data repeats for each singular evaluation.

Makes sense to make a dataset trying to get a 1:1 relation between companies and reviews tables for analysis and also see how many of the companies listed actually have reviews

In [29]:
#Looking for the companies with reviews that are also in the companies tables

#I'm using aggregation here because it makes safe to assume that there will be only one row for each company
df_aggregated_reviews = df_reviews[['google_id','reviews','rating']].groupby(by='google_id').agg({
    'reviews': 'max',   # Take the maximum value for 'reviews'
    'rating': 'mean'    # Take the mean for 'rating'
})

In [31]:
df_aggregated_reviews.reset_index(inplace=True)

In [32]:
df_aggregated_reviews

,google_id,reviews,rating
0,0x54900fa7f798bdd1:0x778e5ec2841c59e8,31.0,4.7
1,0x5490117e221f6117:0x9e5eea3090226dc5,43.0,5.0
2,0x549013daa18732b5:0xed1239c5aef8a340,39.0,5.0
3,0x549013ff5c745f65:0x849cdc83bcbe9694,100.0,4.8
4,0x549013ff84f9de59:0x4e1ff815e97cc509,8.0,3.5
...,...,...,...
81,0x88d9c1f6c4b56083:0x34e94d8a2b0ebfc9,7.0,5.0
82,0x88d9c3ff0a978399:0xc7d67b9452411c0,220.0,4.9
83,0x88d9c75d57aa65e5:0x7741d2b54e949724,83.0,4.9
84,0x88d9cbd3295fd601:0xb54a67564654b9a4,37.0,5.0


In [33]:
#Now let's see how many are found on the companies dataset
df_merged = pd.merge(left=df_aggregated_reviews, right=df_google, on='google_id', how='inner')

In [36]:
df_merged.shape

(86, 55)

86 rows means that all reviewed companies are also in the companies dataframe